In [1]:
from src.training.ModelTrainer import ModelTrainer
from src.datasets.shape_net.shape_net_v3_sdf import ShapeNetV3SDF
from src.utils.util import seed_all
from src.utils.visualizations import visualize_sdf_as_mesh
from tqdm.notebook import tqdm as tqdm
import numpy as np
import torch
%load_ext autoreload
%load_ext tensorboard
%autoreload 2
seed_all(111)
mem = torch.cuda.mem_get_info()
print("Memory Before: %s", mem)
trainer = ModelTrainer(dataset_type=ShapeNetV3SDF,
                       options={"tdm_notebook": True})

dataset = trainer.data_loader_handler.dataset
print("Dataset length: ", len(dataset))
mem = torch.cuda.mem_get_info()
print("Memory After: %s", mem)
model = trainer.model


Memory Before: %s (10120527872, 10385358848)
Using device: cuda:0
{'name': 'presentationModelComparison', 'description': 'plzWork2', 'experiment_id': 'Dev', 'extra_notes': 'none', 'logs_dir': 'logs', 'is_train': True, 'device': 'cuda:0', 'batch_size': 16, 'num_workers': 20, 'test_size': 0.1, 'n_epochs': 50000, 'append_loss_every': 50, 'print_every': 10, 'validate_every': 1480, 'save_every': 1480, 'save_every_nepochs': 5, 'start_epoch': 0, 'start_iteration': 0, 'visualize_every': 200, 'apply_metrics_every': 1480, 'load_ckpt': True, 'ckpt_path': 'logs/GlobalPVQVAE/FullTrain/2024_06_03_12_37_19/checkpoints/epoch-latest.ckpt', 'use_scheduler': True, 'apply_metrics_batch_count': 5}


- Creating new directory logs/presentationModelComparison/Dev/2024_06_05_22_41_45
- Creating new directory logs/presentationModelComparison/Dev/2024_06_05_22_41_45/checkpoints
- Creating new directory logs/presentationModelComparison/Dev/2024_06_05_22_41_45/tb
- Creating new directory logs/presentationModelComparison/Dev/2024_06_05_22_41_45/visuals
- Creating new directory logs/presentationModelComparison/Dev/2024_06_05_22_41_45/modelsummary
/home/youssef/.cache/pypoetry/virtualenvs/vqgan_scene_reconstruction-cT5po-tB-py3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[*] Enc has Attn at i_level, i_block: 3, 0
Decoding of shape (1, 64, 8, 8, 8) = 32768 dimensions.
[*] Dec has Attn at i_level, i_block: 3, 0
Using device: cuda:0
Initializing model weights with normal initialization
Model loaded from logs/GlobalPVQVAE/FullTrain/2024_06_03_12_37_19/checkpoints/epoch-latest.ckpt
Model size: 99.591MB
Total_params: 26.1M
Dataset length:  6579
Memory After: %s (6695878656, 10385358848)


In [2]:
from src.training.SDFVisualizer import SDFVisualizer
from src.metrics.pytorch_3d_chamfer_dist import Pytorch3DChamferDistance
from src.metrics.signed_iou import SignedIou
from src.metrics.pytorch_3d_chamfer_dist import Pytorch3DChamferDistance
from src.metrics.signed_iou import SignedIou
chm = Pytorch3DChamferDistance()
iou = SignedIou()
path = "./src/testVisuals/autoEncoder/"
data_loader = trainer.data_loader_handler.validation_dataloader
with torch.no_grad():
    chamferDistTotal = 0
    iouTotal = 0
    i = 0
    for index, batch in tqdm(enumerate(data_loader), total=len(data_loader)):
        ShapeNetV3SDF.move_batch_to_device(batch, "cuda:0")
        sdfs = batch['sdf']
        pred = model.inference(sdfs)
        visualizer = SDFVisualizer("cuda:0",path,"reconst", 0, index)
        visualizer.visualize(sdfs)
        chamferDistTotal += chm.calc_batch(pred,sdfs)
        iouTotal += iou.calc_batch(pred, sdfs)
        i+=1
chamferDistTotal =  chamferDistTotal/(i)
iouTotal = iouTotal/(i)

/home/youssef/.cache/pypoetry/virtualenvs/vqgan_scene_reconstruction-cT5po-tB-py3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/41 [00:00<?, ?it/s]

./src/testVisuals/autoEncoder//reconst/epoch_0_iter_0.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_1.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_2.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_3.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_4.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_5.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_6.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_7.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_8.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_9.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_10.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_11.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_12.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_13.png saved
./src/testVisuals/autoEncoder//reconst/epoch_0_iter_14.png saved
./src/testVisuals/autoEncoder//reco

In [ ]:
iouTotal

In [ ]:
chamferDistTotal